# Code

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px


class Person:
    def __init__(self, name: str, base_will_power: int):
        self.name = name
        self.base_will_power = base_will_power
        self.current_life_score = 50

    def calc_new_life_score(self, decision_dict: dict[str, int]) -> float:
        good_decision_threshold = (
            self.base_will_power
            + 0.01 * decision_dict["easy_decision_score"]
            + 0.01 * self.current_life_score
        )
        if good_decision_threshold >= decision_dict["hard_decision_score"]:
            self.current_life_score += decision_dict["hard_decision_score"] * 0.01
        else:
            self.current_life_score += decision_dict["easy_decision_score"] * 0.01
        self.current_life_score = np.clip(self.current_life_score, 0, 100)
        return self.current_life_score

    def update_base_will_power(self, new_base_will_power: int):
        self.base_will_power = new_base_will_power


class LifeSimulation:
    def __init__(self, people: list[Person]):
        self.people = people

    def add_person(self, person: Person):
        self.people.append(person)

    def get_people(self) -> list[Person]:
        return self.people

    def get_person_by_name(self, name: str) -> Person | None:
        for person in self.people:
            if person.name == name:
                return person
        return None

    def get_number_of_people(self) -> int:
        return len(self.people)

    def generate_decision_dict(self) -> dict[str, int]:
        hard_decision_score = np.random.randint(0, 100)
        easy_decision_score = np.random.randint(0, 100) * -1
        return {
            "hard_decision_score": hard_decision_score,
            "easy_decision_score": easy_decision_score,
        }

    def simlulate_life(self, number_of_decisions: int):
        life_score_history = {"decision_number": list[int](range(number_of_decisions))}
        for person in self.people:
            life_score_history[person.name] = []
            for decision_number in range(number_of_decisions):
                decision_dict = self.generate_decision_dict()
                person.calc_new_life_score(decision_dict)
                life_score_history[person.name].append(person.current_life_score)
                if decision_number == number_of_decisions / 2:
                    if person.base_will_power <= 50:
                        person.update_base_will_power(75)
        df = pd.DataFrame(life_score_history)
        print(df)
        
        fig = px.line(df, x="decision_number", y=df.columns, title="Life Score History")
        fig.show()


def main():

    people = [
        Person("Amy", 90),
        Person("Barry", 75),
        Person("Cindy", 64),
        Person("Danny", 45),
        Person("Ethan", 25),
        Person("Frank", 10),
    ]
    simulation = LifeSimulation(people)
    simulation.simlulate_life(2000)


if __name__ == "__main__":
    main()

      decision_number     Amy   Barry  Cindy  Danny   Ethan  Frank
0                   0   50.71   50.06  50.44  50.14   50.11  50.00
1                   1   51.42   49.70  49.76  49.83   49.72  49.10
2                   2   51.71   50.01  50.29  50.27   49.79  48.44
3                   3   51.72   49.92  50.80  49.89   49.56  48.43
4                   4   52.60   50.33  50.86  49.82   49.65  48.40
...               ...     ...     ...    ...    ...     ...    ...
1995             1995  100.00  100.00  97.43  99.14  100.00  97.99
1996             1996  100.00  100.00  97.49  98.48  100.00  97.60
1997             1997  100.00  100.00  97.60  98.66  100.00  96.84
1998             1998  100.00  100.00  98.12  98.83  100.00  96.97
1999             1999  100.00   99.38  98.44  99.18  100.00  97.49

[2000 rows x 7 columns]
